<a href="https://colab.research.google.com/github/dhrruvchotai/Current_Learnings/blob/main/Pytorch/fashion_mnist_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
path = kagglehub.dataset_download("zalando-research/fashionmnist")

Using Colab cache for faster access to the 'fashionmnist' dataset.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Deep Learning/Pytorch/fmnist_small.csv")

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [ ]:
df.shape

(6000, 785)

In [ ]:
X = df.drop(columns=["label"])
y = df["label"]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
3897,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5628,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.003922,...,0.356863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1756,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2346,0.0,0.0,0.0,0.0,0.0,0.003922,0.000000,0.000000,0.0,0.000000,...,0.003922,0.000000,0.000000,0.000000,0.000000,0.254902,0.090196,0.000000,0.0,0.0
2996,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.262745,...,0.533333,0.470588,0.286275,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5191,0.0,0.0,0.0,0.0,0.0,0.000000,0.003922,0.000000,0.0,0.200000,...,0.000000,0.000000,0.003922,0.000000,0.031373,0.258824,0.000000,0.000000,0.0,0.0
5226,0.0,0.0,0.0,0.0,0.0,0.000000,0.011765,0.003922,0.0,0.000000,...,0.439216,0.474510,0.474510,0.027451,0.000000,0.003922,0.000000,0.000000,0.0,0.0
5390,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [ ]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features.values,dtype=torch.float32)
    self.labels = torch.tensor(labels.values,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index],self.labels[index]

In [ ]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
class Model(nn.Module):

  def __init__(self,num_features):
    super().__init__()
    self.neural_net = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
        #will not add softmax here because there is softmax function in built in cross entropy loss in pytorch
    )

  def forward(self,features):
    return self.neural_net(features)


In [ ]:
num_features = X_train.shape[1]
model = Model(num_features)


In [ ]:
epochs = 100
learning_rate = 0.1

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),learning_rate)

In [ ]:
num_train_batches = len(train_loader)
num_train_batches

150

In [ ]:
#training loop
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features,batch_labels in train_loader:
    #forward pass
    y_pred = model(batch_features)
    #cal loss
    loss = loss_function(y_pred,batch_labels)
    #clear gradients
    optimizer.zero_grad()
    #backprop
    loss.backward()
    #update params
    optimizer.step()
    total_epoch_loss += loss.item()

  print(f"Epoch {epoch+1} Loss : {total_epoch_loss/num_train_batches}")

Epoch 1 Loss : 1.322568706671397
Epoch 2 Loss : 0.7899165542920431
Epoch 3 Loss : 0.6556014315287272
Epoch 4 Loss : 0.5857545258601506
Epoch 5 Loss : 0.5399637194474538
Epoch 6 Loss : 0.4999191439151764
Epoch 7 Loss : 0.46561806817849477
Epoch 8 Loss : 0.4463784702618917
Epoch 9 Loss : 0.4244719104965528
Epoch 10 Loss : 0.4043931700785955
Epoch 11 Loss : 0.3886548100908597
Epoch 12 Loss : 0.3658471514284611
Epoch 13 Loss : 0.35921288733681045
Epoch 14 Loss : 0.3400715800126394
Epoch 15 Loss : 0.3245877398053805
Epoch 16 Loss : 0.33031851331392925
Epoch 17 Loss : 0.3064374203483264
Epoch 18 Loss : 0.2959343317647775
Epoch 19 Loss : 0.2793607316662868
Epoch 20 Loss : 0.2757481794555982
Epoch 21 Loss : 0.25758648176987964
Epoch 22 Loss : 0.25278453682859736
Epoch 23 Loss : 0.24509644448757173
Epoch 24 Loss : 0.23279751673340798
Epoch 25 Loss : 0.228377908517917
Epoch 26 Loss : 0.22576218167940776
Epoch 27 Loss : 0.2228279365847508
Epoch 28 Loss : 0.21137356559435527
Epoch 29 Loss : 0.1950

In [ ]:
#To Check how torch.max function works
# torch.manual_seed(42)
# outputs = torch.rand(32,10)
# print(outputs[0])

In [ ]:
# _, output = torch.max(outputs,dim=1)
# print(f"{_} and {output}")

tensor([0.9593, 0.9346, 0.8317, 0.9516, 0.9464, 0.9103, 0.9147, 0.9811, 0.8035,
        0.9578, 0.6249, 0.9998, 0.9192, 0.9302, 0.9555, 0.9554, 0.9545, 0.8870,
        0.7628, 0.9297, 0.9211, 0.9837, 0.8788, 0.9208, 0.8087, 0.7797, 0.9696,
        0.9729, 0.9192, 0.6858, 0.9193, 0.9929]) and tensor([3, 0, 3, 0, 3, 3, 6, 2, 4, 6, 1, 0, 8, 0, 8, 4, 0, 8, 5, 3, 2, 5, 6, 3,
        9, 6, 5, 4, 8, 4, 5, 7])


In [ ]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    outputs = model(batch_features)
    _, predicted = torch.max(outputs,dim=1)
    total = total + batch_features.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(correct/total)

0.8358333333333333
